In [1]:
#Import libraries

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Import user-item data

In [4]:
user_item = pd.read_csv("animelists_cleaned.csv")

In [5]:
user_item.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:43:35,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-03 10:53:57,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN


In [6]:
len(user_item)

31284030

In [7]:
#Filter user-item where "my_watched_episodes" > 0

In [8]:
user_item = user_item[user_item["my_watched_episodes"] > 0]

In [9]:
len(user_item)

23137159

In [10]:
#Import anime data

In [11]:
items = pd.read_csv("anime_cleaned.csv")

In [12]:
items.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0,2012.0


In [13]:
len(items)

6668

In [14]:
#Filter items where "type" is not "Music"

In [15]:
items = items[items['type'] != "Music"]

In [16]:
len(items)

6570

In [17]:
#Filter user-item where "anime_id" is in items  

In [18]:
user_item = user_item[user_item["anime_id"].isin(items["anime_id"])]

In [19]:
len(user_item)

23074533

In [20]:
#Filter items where "anime_id" is in user-item

In [21]:
items = items[items["anime_id"].isin(user_item["anime_id"])]

In [22]:
len(items)

6513

In [23]:
#Concatenate genre and studio labels

In [24]:
items['labels'] = items['genre'] + ", " + items['studio']

In [25]:
items[['anime_id','labels']]

,anime_id,labels
0,11013,"Comedy, Supernatural, Romance, Shounen, David ..."
1,2104,"Comedy, Parody, Romance, School, Shounen, Gonzo"
2,5262,"Comedy, Magic, School, Shoujo, Satelight"
3,721,"Comedy, Drama, Magic, Romance, Fantasy, Hal Fi..."
4,12365,"Comedy, Drama, Romance, Shounen, J.C.Staff"
...,...,...
6652,37858,"Comedy, Kyoto Animation"
6656,37667,"Horror, Parody, Supernatural, DLE"
6657,37090,"Action, Super Power, Supernatural, Haoliners A..."
6658,36913,"Comedy, OLM"


In [26]:
#Sort items by anime_id and reset index as anime_id

In [27]:
items.sort_values('anime_id', inplace=True)

In [28]:
items.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year,labels
4358,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,"{'Adaptation': [{'mal_id': 173, 'type': 'manga...",Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","['""Tank!"" by The Seatbelts (eps 1-25)']","['#1: ""The Real Folk Blues"" by The Seatbelts f...",24.0,1998.0,"Action, Adventure, Comedy, Drama, Sci-Fi, Spac..."
5216,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ天国の扉,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,"{'Parent story': [{'mal_id': 1, 'type': 'anime...","Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"Action, Space, Drama, Mystery, Sci-Fi","['""Ask DNA"" by The Seatbelts featuring Raju Ra...","['""Gotta Knock a Little Harder"" by The Seatbel...",114.0,2001.0,"Action, Space, Drama, Mystery, Sci-Fi, Bones"
2603,6,Trigun,Trigun,トライガン,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,"{'Adaptation': [{'mal_id': 703, 'type': 'manga...",Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","['""H.T."" by Tsuneo Imahori']","['""Kaze wa Mirai ni Fuku (The Wind Blows to th...",24.0,1998.0,"Action, Sci-Fi, Adventure, Comedy, Drama, Shou..."
4815,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN,WHR,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,[],Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Magic, Police, Supernatural, Drama, My...","['""Shell"" by Bana']","['#1: ""Half Pain"" by Bana (eps 01-14,16-25)', ...",25.0,2002.0,"Action, Magic, Police, Supernatural, Drama, My..."
4436,8,Beet the Vandel Buster,Beet the Vandel Buster,冒険王ビィト,"Adventure King Beet, Bouken Ou Beet",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,52,Finished Airing,...,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang...",NaN,NaN,Toei Animation,"Adventure, Fantasy, Shounen, Supernatural","['#1: ""Emotion"" by Sunbrain', '#2: ""Wish Men"" ...","['#1: ""pureness"" by Nana Kitade (eps 1-13)', '...",23.0,2004.0,"Adventure, Fantasy, Shounen, Supernatural, Toe..."


In [29]:
items.set_index(items['anime_id'],inplace=True)

In [30]:
items.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year,labels
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,"{'Adaptation': [{'mal_id': 173, 'type': 'manga...",Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","['""Tank!"" by The Seatbelts (eps 1-25)']","['#1: ""The Real Folk Blues"" by The Seatbelts f...",24.0,1998.0,"Action, Adventure, Comedy, Drama, Sci-Fi, Spac..."
5,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ天国の扉,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,"{'Parent story': [{'mal_id': 1, 'type': 'anime...","Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"Action, Space, Drama, Mystery, Sci-Fi","['""Ask DNA"" by The Seatbelts featuring Raju Ra...","['""Gotta Knock a Little Harder"" by The Seatbel...",114.0,2001.0,"Action, Space, Drama, Mystery, Sci-Fi, Bones"
6,6,Trigun,Trigun,トライガン,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,"{'Adaptation': [{'mal_id': 703, 'type': 'manga...",Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","['""H.T."" by Tsuneo Imahori']","['""Kaze wa Mirai ni Fuku (The Wind Blows to th...",24.0,1998.0,"Action, Sci-Fi, Adventure, Comedy, Drama, Shou..."
7,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN,WHR,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,[],Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Magic, Police, Supernatural, Drama, My...","['""Shell"" by Bana']","['#1: ""Half Pain"" by Bana (eps 01-14,16-25)', ...",25.0,2002.0,"Action, Magic, Police, Supernatural, Drama, My..."
8,8,Beet the Vandel Buster,Beet the Vandel Buster,冒険王ビィト,"Adventure King Beet, Bouken Ou Beet",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,52,Finished Airing,...,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang...",NaN,NaN,Toei Animation,"Adventure, Fantasy, Shounen, Supernatural","['#1: ""Emotion"" by Sunbrain', '#2: ""Wish Men"" ...","['#1: ""pureness"" by Nana Kitade (eps 1-13)', '...",23.0,2004.0,"Adventure, Fantasy, Shounen, Supernatural, Toe..."


In [31]:
#Tf-idf vectorize labels

In [32]:
vect = TfidfVectorizer()

In [33]:
labels = vect.fit_transform(items['labels'].values.astype('U'))

In [34]:
labels

<6513x607 sparse matrix of type '<class 'numpy.float64'>'
	with 38072 stored elements in Compressed Sparse Row format>

In [35]:
#Convert sparse matrix to 2D array

In [36]:
labels = labels.toarray()

In [37]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
#Build cosine similarity matrix for labels

In [39]:
cos_sim_labels = cosine_similarity(labels)

In [40]:
cos_sim_labels.shape

(6513, 6513)

In [41]:
#Pivot user-item lookup table to user-item network table 

In [42]:
user_item_network = user_item.pivot_table(index='anime_id',columns='username',values='my_watched_episodes',aggfunc = 'count').fillna(0)

In [43]:
user_item_network.head()

username,----phoebelyn,---L-AND-AME-4EV,--AnimeBoy--,--Etsuko--,--FallenAngel--,--Kin--,--N--,--PHOENIX--,--RIE--,--Sc--,...,zzeldaxx7,zzeroparticle,zzganz,zzrorozz,zzs,zzshinzozz,zzvl,zzz275,zzzcielo,zzzzz-chan
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
#Build cosine similarity matrix for user-item network table

In [45]:
cos_sim_network = cosine_similarity(user_item_network)

In [46]:
cos_sim_network.shape

(6513, 6513)

In [47]:
#Take Hadamard product of cosine similarity matrices  

In [48]:
cos_sim = cos_sim_network * cos_sim_labels

In [49]:
cos_sim.shape

(6513, 6513)

In [50]:
#Transform matrix into dataframe - reindex rows and columns using anime_id

In [51]:
cosine_table = pd.DataFrame(cos_sim, index = list(user_item_network.index))
cosine_table.columns = list(user_item_network.index)

In [52]:
cosine_table.head()

,1,5,6,7,8,15,16,17,18,19,...,37672,37678,37710,37719,37790,37802,37814,37858,37859,37860
1,1.000000,0.348131,0.327750,0.122724,0.012154,0.027968,0.042242,0.005262,0.038397,0.030850,...,0.001994,0.000074,0.0,0.007921,0.0,0.0,0.0,0.000276,0.000276,0.000546
5,0.348131,1.000000,0.176286,0.096619,0.000000,0.010592,0.021239,0.000000,0.029198,0.064175,...,0.002279,0.000000,0.0,0.003101,0.0,0.0,0.0,0.000000,0.000000,0.000000
6,0.327750,0.176286,1.000000,0.057189,0.033356,0.064224,0.041378,0.019979,0.042662,0.102132,...,0.002007,0.000000,0.0,0.006199,0.0,0.0,0.0,0.000547,0.000547,0.000576
7,0.122724,0.096619,0.057189,1.000000,0.018238,0.009864,0.018235,0.000000,0.020932,0.096394,...,0.001095,0.000000,0.0,0.001441,0.0,0.0,0.0,0.000000,0.000000,0.000000
8,0.012154,0.000000,0.033356,0.018238,1.000000,0.020936,0.000000,0.034577,0.000000,0.000000,...,0.001496,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.002103


In [53]:
#Save anime lookup and cosine sim table objects 

In [54]:
items.to_csv('anime_lookup_table.csv')

In [55]:
cosine_table.to_csv('cosine_table.csv')